#### Test of stuff exported from Mathematica for case "MatrixElementExport" of CommonPlatformTriple_4B4W_2x4B4W4W

## Preliminaries

In [3]:
import sys
sys.path.append('/Users/t.seetharamu.1/Documents/svn/GCIF/GCIF planning/Infrastructure cryogenics/CommonPlatformTriple_4B4W_2x4B4W4W/MatrixElementExport')

### Other system dependencies

In [4]:
import os

In [5]:
from math import pi,sqrt

In [6]:
import numpy as np

In [7]:
# import scipy.linalg

In [8]:
import control as ct

In [9]:
import matplotlib.pyplot as plt

In [10]:
import os
import pandas as pd
import inspect
from math import pi,sqrt
from decimal import Decimal

import numpy as np

# import scipy.linalg
from matplotlib.ticker import MultipleLocator
import control as ct

import matplotlib.pyplot as plt
import importlib

import matplotlib
matplotlib.rcParams['font.sans-serif'] = "Comic Sans MS"
# Then, "ALWAYS use sans-serif fonts"
matplotlib.rcParams['font.family'] = "sans-serif"
plt.rcParams['font.size'] = 18          # Default font size for text
plt.rcParams['axes.titlesize'] = 22     # Font size for axes titles
plt.rcParams['axes.labelsize'] = 18     # Font size for axes labels
plt.rcParams['xtick.labelsize'] = 14    # Font size for x-axis tick labels
plt.rcParams['ytick.labelsize'] = 14    # Font size for y-axis tick labels
plt.rcParams['legend.fontsize'] = 15    # Font size for legend


### CommonPlatformTriple_4B4W_2x4B4W4W model definition data and matrix element functions exported from Mathematica 

In [11]:
import symbexportCommonPlatformTriple_4B4W_2x4B4W4Wfull as tpmodel

### Parameters exported from Mathematica

In [12]:
import CommonPlatformTriple_4B4W_2x4B4W4W_MatrixElementExport_cptriple as tp

In [13]:
n3 = 0.0405
def reboot_params():
    importlib.reload(tp)
    tp.n3 = 0.0405
    tp.n4 = n3
    tp.n5 = n3
    revised_sus_length = 0.298
    tp.kw1= 1396.07
    tp.kw2=tp.kw1
    tp.kw3=70520.4
    tp.l1 = revised_sus_length
    tp.l2 = revised_sus_length
    tp.l3 = revised_sus_length

In [14]:
def original_params():
    importlib.reload(tp)
    revised_sus_length = 0.298


    data =     [["Masses", "m1 / m2 / m3", [round(tp.m1,2) ,round(tp.m2,2) ,round(tp.m3,2)]],

    ["Roll Moment of Inertia", "I1,x / I2,x / I3,x",  [round(i,5) for i in [tp.I1x, tp.I2x , tp.I3x]]],

    ["Pitch Moment of Inertia", "I1,y / I2,y / I3,y",  [round(i,5) for i in [tp.I1y, tp.I2y , tp.I3y]]],

    ["Yaw Moment of Inertia", "I1,z / I2,z / I3,z", [round(i,5) for i in [tp.I1z, tp.I2z , tp.I3z]]],
    
            ["Stretched wire lengths", "l1 / l2 / l3", [tp.l1 , tp.l2, tp.l3]], 
            ["Upper end wire vertical attachment distance", "dtop / d1/ d3 " , [round(i,3) for i in [tp.dtop, tp.d1 , tp.d3]]],
            ["Bottom end wire vertical attachment distance", "d0 / d2 / d4" , [round(i,3) for i in [tp.d0 , tp.d2, tp.d4]]],
            
            ["Longitudinal wire separation" , "su / si / sl", [tp.su, tp.si , tp.sl]],
            ["Upper transverse wire separation", "n0 / n2 / n4 ", [tp.n0, tp.n2 , tp.n4]],
            ["Bottom transverse wire separation", "n1 / n3 / n5 ", [tp.n1, tp.n3 , tp.n5]],
            ["Vertical Spring stiffness", "kbuz / kbiz / kblz", [tp.kbuz, "NA", tp.kblz]],
            ["Wire Lengths", "l1 / l2 / l3", [tp.l1 , tp.l2 , tp.l3]],
            ["Youngs Moduli of the wires", "Y1 / Y2 / Y3", ['%.5E' % Decimal(tp.Y1), '%.5E' % Decimal(tp.Y2), '%.5E' % Decimal(tp.Y3)]]]



    params = pd.DataFrame(data, columns = ["Description", "Parameter","Values"])
    # Setting display options
    pd.set_option('display.width', 1000)  # Set width to a large value to prevent wrapping
    pd.set_option('display.max_colwidth', None)  # Display full contents of each cell without truncation
    print(params)

In [71]:
def print_modes(eigens):
    Hz = eigens[0]
    evecs = eigens[1]
    dofs = []
    dof_amps = []
    for j in range(0,len(Hz)): 
        df =pd.DataFrame(np.array(list(zip(tpmodel.allvars,list(evecs[j])))), columns=["DOF","Amp"])
        df["Amp"] = [(round(float(i),2)) for i in list(df["Amp"])] 
        df = (df[df["Amp"]!=0])
        df = df.sort_values(by='Amp', ascending=False, key=abs)
        dofs.append(np.array(df["DOF"]))
        dof_amps.append(np.array(df["Amp"]))

    modes = pd.DataFrame({"Freq":Hz, "DOF":dofs, "Amps":dof_amps})
    
    def calculate_modevis(row):
        dof = row['DOF']
        amps = row['Amps']
        
        # Identify the mode type based on the first entry in DOF
        mode_type = dof[0].rstrip('123IE')  # e.g., "roll", "pitch"
        # print (mode_type)
        # Find indices of the first and third components for I and E objects
        first_idx = next((i for i, d in enumerate(dof) if d == f"{mode_type}1"), None)
        print (first_idx)
        third_I_idx = next((i for i, d in enumerate(dof) if d == f"{mode_type}3I"), None)
        print (third_I_idx)
        third_E_idx = next((i for i, d in enumerate(dof) if d == f"{mode_type}3E"), None)
        print (third_E_idx)
        # Calculate the ratio if both indices are found for I and E objects
        ratio_I = amps[first_idx] / amps[third_I_idx] if first_idx is not None and third_I_idx is not None and amps[third_I_idx] != 0 else None
        ratio_E = amps[first_idx] / amps[third_E_idx] if first_idx is not None and third_E_idx is not None and amps[third_E_idx] != 0 else None
        
        return (ratio_I, ratio_E)


    # Apply the function and create a new column
    modes['Mode_vis'] = modes.apply(calculate_modevis, axis=1)

# # Display the updated dataframe
# display(df)
    return modes

In [16]:
def mode_forest(mode_amp, fig=None, ax = None):
    Hz = mode_amp[0]
    evecs = mode_amp[1]
    dofs = []
    dof_amps = []
    for j in range(0,18): 
        df =pd.DataFrame(np.array(list(zip(tpmodel.allvars,list(evecs[j])))), columns=["DOF","Amp"])
        df["Amp"] = [np.abs(round(float(i),2)) for i in list(df["Amp"])] 
        df = (df[df["Amp"]!=0])
        df = df.sort_values(by='Amp', ascending=False)
        dofs.append(np.array(df["DOF"]))
        dof_amps.append(np.array(df["Amp"]))
        
    
    
        if any(df["DOF"].iloc[i] in ["x1", "x2", "x3", "pitch1", "pitch2", "pitch3"] and df["Amp"].iloc[i] > 0.1 for i in range(len(df))):
            ax.scatter(Hz[j],"Long-pitch Mode", c= "red")
            ax.text(Hz[j] + 0.1,"Long-pitch Mode", str(round(Hz[j],3)), fontsize=10)  # Adjust 0.1 to position the text appropriately
        elif any(df["DOF"].iloc[j] in ["y1", "y2", "y3","roll1", "roll2", "roll3"] and df["Amp"].iloc[j] > 0.1 for j in range(len(df))):
            ax.scatter(Hz[j],"Trans-roll Mode", c = "blue")
            ax.text(Hz[j] + 0.1,"Trans-roll Mode", str(round(Hz[j],3)), fontsize=15)
        elif any(df["DOF"].iloc[j] in ["z1", "z2", "z3"] and df["Amp"].iloc[j] > 0.1 for j in range(len(df))):
            ax.scatter(Hz[j],"Bounce Mode", c = "black")    
            ax.text(Hz[j] + 0.1,"Bounce Mode", str(round(Hz[j],3)), fontsize=15)
        elif any(df["DOF"].iloc[j] in ["yaw1", "yaw2", "yaw3"] and df["Amp"].iloc[j] > 0.1 for j in range(len(df))):
            ax.scatter(Hz[j],"Yaw Mode", c = "green")    
            ax.text(Hz[j] + 0.1,"Yaw Mode", str(round(Hz[j],3)), fontsize=15)

        ax.grid()
        ax.set_ylabel("Modes", fontsize = 20)
        ax.set_xlabel("Freq (Hz)", fontsize = 20)
        ax.tick_params(axis = "both", labelsize = 15)
        ax.xaxis.set_minor_locator(MultipleLocator(10))
        
    modes = pd.DataFrame({"Freq":Hz, "DOF":dofs, "Amps":dof_amps})
        
    return fig, ax
         

In [17]:
def changes(mutation):
    reboot_params()
    for name, value in zip(mutation.iloc[:, 0], mutation.iloc[:, 1]):
        locals()[name] = value
    # print(mutation)
    # print(d2)
    revised_sus_length = 0.298
    tp.kw1= 1396.07
    tp.kw2=tp.kw1
    tp.kw3=70520.4
    tp.l1 = revised_sus_length
    tp.l2 = revised_sus_length
    tp.l3 = revised_sus_length
    #Some constraints 
    global n3
    n3 = 0.0405
    n4 = n3
    n5 = n3

        
    def assign_to_tp(mutation, tp):
        for index, row in mutation.iterrows():
            string = row['Parameters']
            value = row['Values']
            # Dynamically set the attribute to the corresponding value
            setattr(tp, string, value)

# Call the function
    assign_to_tp(mutation, tp)
    
    
    pend = tpmodel.pendSS(*[getattr(tp, coeff) for coeff in tpmodel.stdcoeffs])
    (Hz,evecs) = tpmodel.eigenmodes(*[getattr(tp, coeff) for coeff in tpmodel.stdcoeffs])
    mode_amp = (Hz,evecs)
    
    data =     [["Masses", "m1 / m2 / m3", [round(tp.m1,2) ,round(tp.m2,2) ,round(tp.m3,2)]],

    ["Roll Moment of Inertia", "I1,x / I2,x / I3,x",[round(i,5) for i in [tp.I1x, tp.I2x , tp.I3x]]],

    ["Pitch Moment of Inertia", "I1,y / I2,y / I3,y", [round(i,5) for i in [tp.I1y, tp.I2y , tp.I3y]]],

    ["Yaw Moment of Inertia", "I1,z / I2,z / I3,z",  [round(i,5) for i in [tp.I1z, tp.I2z , tp.I3z]]],
    
            ["Stretched wire lengths", "l1 / l2 / l3", [tp.l1 , tp.l2, tp.l3]], 
            ["Upper end wire vertical attachment distance", "dtop / d1/ d3 " , [round(i,3) for i in [tp.dtop, tp.d1 , tp.d3]]],
            ["Bottom end wire vertical attachment distance", "d0 / d2 / d4" , [round(i,3) for i in [tp.d0 , tp.d2, tp.d4]]],
            
            ["Longitudinal wire separation" , "su / si / sl", [tp.su, tp.si , tp.sl]],
            ["Upper transverse wire separation", "n0 / n2 / n4 ", [tp.n0, tp.n2 , tp.n4]],
            ["Bottom transverse wire separation", "n1 / n3 / n5 ", [tp.n1, tp.n3 , tp.n5]],
            ["Vertical Spring stiffness", "kbuz / kbiz / kblz", [tp.kbuz, "NA", tp.kblz]],
            ["Wire Lengths", "l1 / l2 / l3", [tp.l1 , tp.l2 , tp.l3]],
            ["Youngs Moduli of the wires", "Y1 / Y2 / Y3", ['%.5E' % Decimal(tp.Y1), '%.5E' % Decimal(tp.Y2), '%.5E' % Decimal(tp.Y3)]]]


    params = pd.DataFrame(data, columns = ["Description", "Parameter","Values"])
    # Setting display options
    pd.set_option('display.width', 1000)  # Set width to a large value to prevent wrapping
    pd.set_option('display.max_colwidth', None)  # Display full contents of each cell without truncation
    return params, pend, mode_amp



In [18]:
def trimio_tf(pend, newinputs, newoutputs):
	'''A function to trim a state space object to have only the specified subsets of inputs and output.'''
	A=pend.A
	B=pend.B
	C=pend.C
	D=pend.D
	oldinputs=pend.input_index
	oldoutputs=pend.output_index
	oldstates=pend.state_index
	newinputlist = []
	for i in range(len(newinputs)):
			newinputlist.append(oldinputs.get(newinputs[i],0))
	newoutputlist = []
	for i in range(len(newoutputs)):
			newoutputlist.append(oldoutputs.get(newoutputs[i],0))
	BB=B[range(len(oldstates))][...,newinputlist]
	CC=C[newoutputlist]
	DD=D[newoutputlist][...,newinputlist]
	return ct.ss2tf(A,BB,CC,DD)

### The stiffness matrix for the floats

In [19]:
qm = tpmodel.qm(tp.g,tp.m1,tp.m2I,tp.m2E,tp.m3I,tp.m3E,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2Ix,tp.I2Iy,tp.I2Iz,tp.I2Ixy,tp.I2Iyz,tp.I2Izx,tp.I2Ex,tp.I2Ey,tp.I2Ez,tp.I2Exy,tp.I2Eyz,tp.I2Ezx,tp.I3Ix,tp.I3Iy,tp.I3Iz,tp.I3Ixy,tp.I3Iyz,tp.I3Izx,tp.I3Ex,tp.I3Ey,tp.I3Ez,tp.I3Exy,tp.I3Eyz,tp.I3Ezx,tp.dtop,tp.d0,tp.d1I,tp.d1E,tp.d2I,tp.d2E,tp.d3I,tp.d3E,tp.d4I,tp.d4E,tp.chainIx,tp.chainEx,tp.n0,tp.n1,tp.n2I,tp.n2E,tp.n3I,tp.n3E,tp.n4I,tp.n4E,tp.n5I,tp.n5E,tp.su,tp.siI,tp.siE,tp.slI,tp.slE,tp.kbuz,tp.kbizI,tp.kbizE,tp.l1,tp.l2I,tp.l2E,tp.l3I,tp.l3E,tp.kw1,tp.kw2I,tp.kw2E,tp.kw3I,tp.kw3E,tp.Y1,tp.Y2I,tp.Y2E,tp.Y3I,tp.Y3E,tp.M11,tp.M21I,tp.M21E,tp.M31I,tp.M31E)

In [20]:
qm.shape

(12, 12)

### The coupling matrix between floats and params

In [21]:
cqsm = tpmodel.cqsm(tp.g,tp.m1,tp.m2I,tp.m2E,tp.m3I,tp.m3E,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2Ix,tp.I2Iy,tp.I2Iz,tp.I2Ixy,tp.I2Iyz,tp.I2Izx,tp.I2Ex,tp.I2Ey,tp.I2Ez,tp.I2Exy,tp.I2Eyz,tp.I2Ezx,tp.I3Ix,tp.I3Iy,tp.I3Iz,tp.I3Ixy,tp.I3Iyz,tp.I3Izx,tp.I3Ex,tp.I3Ey,tp.I3Ez,tp.I3Exy,tp.I3Eyz,tp.I3Ezx,tp.dtop,tp.d0,tp.d1I,tp.d1E,tp.d2I,tp.d2E,tp.d3I,tp.d3E,tp.d4I,tp.d4E,tp.chainIx,tp.chainEx,tp.n0,tp.n1,tp.n2I,tp.n2E,tp.n3I,tp.n3E,tp.n4I,tp.n4E,tp.n5I,tp.n5E,tp.su,tp.siI,tp.siE,tp.slI,tp.slE,tp.kbuz,tp.kbizI,tp.kbizE,tp.l1,tp.l2I,tp.l2E,tp.l3I,tp.l3E,tp.kw1,tp.kw2I,tp.kw2E,tp.kw3I,tp.kw3E,tp.Y1,tp.Y2I,tp.Y2E,tp.Y3I,tp.Y3E,tp.M11,tp.M21I,tp.M21E,tp.M31I,tp.M31E)

In [22]:
cqsm.shape

(12, 6)

### The coupling matrix between floats and vars

In [23]:
cqxm = tpmodel.cqxm(tp.g,tp.m1,tp.m2I,tp.m2E,tp.m3I,tp.m3E,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2Ix,tp.I2Iy,tp.I2Iz,tp.I2Ixy,tp.I2Iyz,tp.I2Izx,tp.I2Ex,tp.I2Ey,tp.I2Ez,tp.I2Exy,tp.I2Eyz,tp.I2Ezx,tp.I3Ix,tp.I3Iy,tp.I3Iz,tp.I3Ixy,tp.I3Iyz,tp.I3Izx,tp.I3Ex,tp.I3Ey,tp.I3Ez,tp.I3Exy,tp.I3Eyz,tp.I3Ezx,tp.dtop,tp.d0,tp.d1I,tp.d1E,tp.d2I,tp.d2E,tp.d3I,tp.d3E,tp.d4I,tp.d4E,tp.chainIx,tp.chainEx,tp.n0,tp.n1,tp.n2I,tp.n2E,tp.n3I,tp.n3E,tp.n4I,tp.n4E,tp.n5I,tp.n5E,tp.su,tp.siI,tp.siE,tp.slI,tp.slE,tp.kbuz,tp.kbizI,tp.kbizE,tp.l1,tp.l2I,tp.l2E,tp.l3I,tp.l3E,tp.kw1,tp.kw2I,tp.kw2E,tp.kw3I,tp.kw3E,tp.Y1,tp.Y2I,tp.Y2E,tp.Y3I,tp.Y3E,tp.M11,tp.M21I,tp.M21E,tp.M31I,tp.M31E)

In [24]:
cqxm.shape

(12, 30)

### The stiffness matrix for the params (raw, then effective)

In [25]:
sm = tpmodel.sm(tp.g,tp.m1,tp.m2I,tp.m2E,tp.m3I,tp.m3E,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2Ix,tp.I2Iy,tp.I2Iz,tp.I2Ixy,tp.I2Iyz,tp.I2Izx,tp.I2Ex,tp.I2Ey,tp.I2Ez,tp.I2Exy,tp.I2Eyz,tp.I2Ezx,tp.I3Ix,tp.I3Iy,tp.I3Iz,tp.I3Ixy,tp.I3Iyz,tp.I3Izx,tp.I3Ex,tp.I3Ey,tp.I3Ez,tp.I3Exy,tp.I3Eyz,tp.I3Ezx,tp.dtop,tp.d0,tp.d1I,tp.d1E,tp.d2I,tp.d2E,tp.d3I,tp.d3E,tp.d4I,tp.d4E,tp.chainIx,tp.chainEx,tp.n0,tp.n1,tp.n2I,tp.n2E,tp.n3I,tp.n3E,tp.n4I,tp.n4E,tp.n5I,tp.n5E,tp.su,tp.siI,tp.siE,tp.slI,tp.slE,tp.kbuz,tp.kbizI,tp.kbizE,tp.l1,tp.l2I,tp.l2E,tp.l3I,tp.l3E,tp.kw1,tp.kw2I,tp.kw2E,tp.kw3I,tp.kw3E,tp.Y1,tp.Y2I,tp.Y2E,tp.Y3I,tp.Y3E,tp.M11,tp.M21I,tp.M21E,tp.M31I,tp.M31E)

In [26]:
sm.shape

(6, 6)

In [27]:
smeff = tpmodel.smeff(tp.g,tp.m1,tp.m2I,tp.m2E,tp.m3I,tp.m3E,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2Ix,tp.I2Iy,tp.I2Iz,tp.I2Ixy,tp.I2Iyz,tp.I2Izx,tp.I2Ex,tp.I2Ey,tp.I2Ez,tp.I2Exy,tp.I2Eyz,tp.I2Ezx,tp.I3Ix,tp.I3Iy,tp.I3Iz,tp.I3Ixy,tp.I3Iyz,tp.I3Izx,tp.I3Ex,tp.I3Ey,tp.I3Ez,tp.I3Exy,tp.I3Eyz,tp.I3Ezx,tp.dtop,tp.d0,tp.d1I,tp.d1E,tp.d2I,tp.d2E,tp.d3I,tp.d3E,tp.d4I,tp.d4E,tp.chainIx,tp.chainEx,tp.n0,tp.n1,tp.n2I,tp.n2E,tp.n3I,tp.n3E,tp.n4I,tp.n4E,tp.n5I,tp.n5E,tp.su,tp.siI,tp.siE,tp.slI,tp.slE,tp.kbuz,tp.kbizI,tp.kbizE,tp.l1,tp.l2I,tp.l2E,tp.l3I,tp.l3E,tp.kw1,tp.kw2I,tp.kw2E,tp.kw3I,tp.kw3E,tp.Y1,tp.Y2I,tp.Y2E,tp.Y3I,tp.Y3E,tp.M11,tp.M21I,tp.M21E,tp.M31I,tp.M31E)

In [28]:
smeff.shape

(6, 6)

### The coupling matrix between vars and params (raw then effective)

In [29]:
cxsm = tpmodel.cxsm(tp.g,tp.m1,tp.m2I,tp.m2E,tp.m3I,tp.m3E,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2Ix,tp.I2Iy,tp.I2Iz,tp.I2Ixy,tp.I2Iyz,tp.I2Izx,tp.I2Ex,tp.I2Ey,tp.I2Ez,tp.I2Exy,tp.I2Eyz,tp.I2Ezx,tp.I3Ix,tp.I3Iy,tp.I3Iz,tp.I3Ixy,tp.I3Iyz,tp.I3Izx,tp.I3Ex,tp.I3Ey,tp.I3Ez,tp.I3Exy,tp.I3Eyz,tp.I3Ezx,tp.dtop,tp.d0,tp.d1I,tp.d1E,tp.d2I,tp.d2E,tp.d3I,tp.d3E,tp.d4I,tp.d4E,tp.chainIx,tp.chainEx,tp.n0,tp.n1,tp.n2I,tp.n2E,tp.n3I,tp.n3E,tp.n4I,tp.n4E,tp.n5I,tp.n5E,tp.su,tp.siI,tp.siE,tp.slI,tp.slE,tp.kbuz,tp.kbizI,tp.kbizE,tp.l1,tp.l2I,tp.l2E,tp.l3I,tp.l3E,tp.kw1,tp.kw2I,tp.kw2E,tp.kw3I,tp.kw3E,tp.Y1,tp.Y2I,tp.Y2E,tp.Y3I,tp.Y3E,tp.M11,tp.M21I,tp.M21E,tp.M31I,tp.M31E)

In [30]:
cxsm.shape

(30, 6)

In [31]:
cxsmeff = tpmodel.cxsmeff(tp.g,tp.m1,tp.m2I,tp.m2E,tp.m3I,tp.m3E,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2Ix,tp.I2Iy,tp.I2Iz,tp.I2Ixy,tp.I2Iyz,tp.I2Izx,tp.I2Ex,tp.I2Ey,tp.I2Ez,tp.I2Exy,tp.I2Eyz,tp.I2Ezx,tp.I3Ix,tp.I3Iy,tp.I3Iz,tp.I3Ixy,tp.I3Iyz,tp.I3Izx,tp.I3Ex,tp.I3Ey,tp.I3Ez,tp.I3Exy,tp.I3Eyz,tp.I3Ezx,tp.dtop,tp.d0,tp.d1I,tp.d1E,tp.d2I,tp.d2E,tp.d3I,tp.d3E,tp.d4I,tp.d4E,tp.chainIx,tp.chainEx,tp.n0,tp.n1,tp.n2I,tp.n2E,tp.n3I,tp.n3E,tp.n4I,tp.n4E,tp.n5I,tp.n5E,tp.su,tp.siI,tp.siE,tp.slI,tp.slE,tp.kbuz,tp.kbizI,tp.kbizE,tp.l1,tp.l2I,tp.l2E,tp.l3I,tp.l3E,tp.kw1,tp.kw2I,tp.kw2E,tp.kw3I,tp.kw3E,tp.Y1,tp.Y2I,tp.Y2E,tp.Y3I,tp.Y3E,tp.M11,tp.M21I,tp.M21E,tp.M31I,tp.M31E)

In [32]:
cxsmeff.shape

(30, 6)

### The stiffness matrix for vars (raw, then effective)

In [33]:
xm = tpmodel.xm(tp.g,tp.m1,tp.m2I,tp.m2E,tp.m3I,tp.m3E,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2Ix,tp.I2Iy,tp.I2Iz,tp.I2Ixy,tp.I2Iyz,tp.I2Izx,tp.I2Ex,tp.I2Ey,tp.I2Ez,tp.I2Exy,tp.I2Eyz,tp.I2Ezx,tp.I3Ix,tp.I3Iy,tp.I3Iz,tp.I3Ixy,tp.I3Iyz,tp.I3Izx,tp.I3Ex,tp.I3Ey,tp.I3Ez,tp.I3Exy,tp.I3Eyz,tp.I3Ezx,tp.dtop,tp.d0,tp.d1I,tp.d1E,tp.d2I,tp.d2E,tp.d3I,tp.d3E,tp.d4I,tp.d4E,tp.chainIx,tp.chainEx,tp.n0,tp.n1,tp.n2I,tp.n2E,tp.n3I,tp.n3E,tp.n4I,tp.n4E,tp.n5I,tp.n5E,tp.su,tp.siI,tp.siE,tp.slI,tp.slE,tp.kbuz,tp.kbizI,tp.kbizE,tp.l1,tp.l2I,tp.l2E,tp.l3I,tp.l3E,tp.kw1,tp.kw2I,tp.kw2E,tp.kw3I,tp.kw3E,tp.Y1,tp.Y2I,tp.Y2E,tp.Y3I,tp.Y3E,tp.M11,tp.M21I,tp.M21E,tp.M31I,tp.M31E)

In [34]:
xm.shape

(30, 30)

In [35]:
xmeff=tpmodel.xmeff(tp.g,tp.m1,tp.m2I,tp.m2E,tp.m3I,tp.m3E,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2Ix,tp.I2Iy,tp.I2Iz,tp.I2Ixy,tp.I2Iyz,tp.I2Izx,tp.I2Ex,tp.I2Ey,tp.I2Ez,tp.I2Exy,tp.I2Eyz,tp.I2Ezx,tp.I3Ix,tp.I3Iy,tp.I3Iz,tp.I3Ixy,tp.I3Iyz,tp.I3Izx,tp.I3Ex,tp.I3Ey,tp.I3Ez,tp.I3Exy,tp.I3Eyz,tp.I3Ezx,tp.dtop,tp.d0,tp.d1I,tp.d1E,tp.d2I,tp.d2E,tp.d3I,tp.d3E,tp.d4I,tp.d4E,tp.chainIx,tp.chainEx,tp.n0,tp.n1,tp.n2I,tp.n2E,tp.n3I,tp.n3E,tp.n4I,tp.n4E,tp.n5I,tp.n5E,tp.su,tp.siI,tp.siE,tp.slI,tp.slE,tp.kbuz,tp.kbizI,tp.kbizE,tp.l1,tp.l2I,tp.l2E,tp.l3I,tp.l3E,tp.kw1,tp.kw2I,tp.kw2E,tp.kw3I,tp.kw3E,tp.Y1,tp.Y2I,tp.Y2E,tp.Y3I,tp.Y3E,tp.M11,tp.M21I,tp.M21E,tp.M31I,tp.M31E)

In [36]:
xmeff.shape

(30, 30)

### The mass matrix

In [37]:
km = tpmodel.km(tp.g,tp.m1,tp.m2I,tp.m2E,tp.m3I,tp.m3E,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2Ix,tp.I2Iy,tp.I2Iz,tp.I2Ixy,tp.I2Iyz,tp.I2Izx,tp.I2Ex,tp.I2Ey,tp.I2Ez,tp.I2Exy,tp.I2Eyz,tp.I2Ezx,tp.I3Ix,tp.I3Iy,tp.I3Iz,tp.I3Ixy,tp.I3Iyz,tp.I3Izx,tp.I3Ex,tp.I3Ey,tp.I3Ez,tp.I3Exy,tp.I3Eyz,tp.I3Ezx,tp.dtop,tp.d0,tp.d1I,tp.d1E,tp.d2I,tp.d2E,tp.d3I,tp.d3E,tp.d4I,tp.d4E,tp.chainIx,tp.chainEx,tp.n0,tp.n1,tp.n2I,tp.n2E,tp.n3I,tp.n3E,tp.n4I,tp.n4E,tp.n5I,tp.n5E,tp.su,tp.siI,tp.siE,tp.slI,tp.slE,tp.kbuz,tp.kbizI,tp.kbizE,tp.l1,tp.l2I,tp.l2E,tp.l3I,tp.l3E,tp.kw1,tp.kw2I,tp.kw2E,tp.kw3I,tp.kw3E,tp.Y1,tp.Y2I,tp.Y2E,tp.Y3I,tp.Y3E,tp.M11,tp.M21I,tp.M21E,tp.M31I,tp.M31E)

In [38]:
km.shape

(30, 30)

# Generate state space

## Generate full A, B, C, D matrices

In [39]:
A = tpmodel.A(tp.g,tp.m1,tp.m2I,tp.m2E,tp.m3I,tp.m3E,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2Ix,tp.I2Iy,tp.I2Iz,tp.I2Ixy,tp.I2Iyz,tp.I2Izx,tp.I2Ex,tp.I2Ey,tp.I2Ez,tp.I2Exy,tp.I2Eyz,tp.I2Ezx,tp.I3Ix,tp.I3Iy,tp.I3Iz,tp.I3Ixy,tp.I3Iyz,tp.I3Izx,tp.I3Ex,tp.I3Ey,tp.I3Ez,tp.I3Exy,tp.I3Eyz,tp.I3Ezx,tp.dtop,tp.d0,tp.d1I,tp.d1E,tp.d2I,tp.d2E,tp.d3I,tp.d3E,tp.d4I,tp.d4E,tp.chainIx,tp.chainEx,tp.n0,tp.n1,tp.n2I,tp.n2E,tp.n3I,tp.n3E,tp.n4I,tp.n4E,tp.n5I,tp.n5E,tp.su,tp.siI,tp.siE,tp.slI,tp.slE,tp.kbuz,tp.kbizI,tp.kbizE,tp.l1,tp.l2I,tp.l2E,tp.l3I,tp.l3E,tp.kw1,tp.kw2I,tp.kw2E,tp.kw3I,tp.kw3E,tp.Y1,tp.Y2I,tp.Y2E,tp.Y3I,tp.Y3E,tp.M11,tp.M21I,tp.M21E,tp.M31I,tp.M31E)

In [40]:
A.shape

(60, 60)

In [41]:
B = tpmodel.B(tp.g,tp.m1,tp.m2I,tp.m2E,tp.m3I,tp.m3E,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2Ix,tp.I2Iy,tp.I2Iz,tp.I2Ixy,tp.I2Iyz,tp.I2Izx,tp.I2Ex,tp.I2Ey,tp.I2Ez,tp.I2Exy,tp.I2Eyz,tp.I2Ezx,tp.I3Ix,tp.I3Iy,tp.I3Iz,tp.I3Ixy,tp.I3Iyz,tp.I3Izx,tp.I3Ex,tp.I3Ey,tp.I3Ez,tp.I3Exy,tp.I3Eyz,tp.I3Ezx,tp.dtop,tp.d0,tp.d1I,tp.d1E,tp.d2I,tp.d2E,tp.d3I,tp.d3E,tp.d4I,tp.d4E,tp.chainIx,tp.chainEx,tp.n0,tp.n1,tp.n2I,tp.n2E,tp.n3I,tp.n3E,tp.n4I,tp.n4E,tp.n5I,tp.n5E,tp.su,tp.siI,tp.siE,tp.slI,tp.slE,tp.kbuz,tp.kbizI,tp.kbizE,tp.l1,tp.l2I,tp.l2E,tp.l3I,tp.l3E,tp.kw1,tp.kw2I,tp.kw2E,tp.kw3I,tp.kw3E,tp.Y1,tp.Y2I,tp.Y2E,tp.Y3I,tp.Y3E,tp.M11,tp.M21I,tp.M21E,tp.M31I,tp.M31E)

In [42]:
B.shape

(60, 36)

In [43]:
C = tpmodel.C(tp.g,tp.m1,tp.m2I,tp.m2E,tp.m3I,tp.m3E,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2Ix,tp.I2Iy,tp.I2Iz,tp.I2Ixy,tp.I2Iyz,tp.I2Izx,tp.I2Ex,tp.I2Ey,tp.I2Ez,tp.I2Exy,tp.I2Eyz,tp.I2Ezx,tp.I3Ix,tp.I3Iy,tp.I3Iz,tp.I3Ixy,tp.I3Iyz,tp.I3Izx,tp.I3Ex,tp.I3Ey,tp.I3Ez,tp.I3Exy,tp.I3Eyz,tp.I3Ezx,tp.dtop,tp.d0,tp.d1I,tp.d1E,tp.d2I,tp.d2E,tp.d3I,tp.d3E,tp.d4I,tp.d4E,tp.chainIx,tp.chainEx,tp.n0,tp.n1,tp.n2I,tp.n2E,tp.n3I,tp.n3E,tp.n4I,tp.n4E,tp.n5I,tp.n5E,tp.su,tp.siI,tp.siE,tp.slI,tp.slE,tp.kbuz,tp.kbizI,tp.kbizE,tp.l1,tp.l2I,tp.l2E,tp.l3I,tp.l3E,tp.kw1,tp.kw2I,tp.kw2E,tp.kw3I,tp.kw3E,tp.Y1,tp.Y2I,tp.Y2E,tp.Y3I,tp.Y3E,tp.M11,tp.M21I,tp.M21E,tp.M31I,tp.M31E)

In [44]:
C.shape

(36, 60)

In [45]:
D = tpmodel.D(tp.g,tp.m1,tp.m2I,tp.m2E,tp.m3I,tp.m3E,tp.I1x,tp.I1y,tp.I1z,tp.I1xy,tp.I1yz,tp.I1zx,tp.I2Ix,tp.I2Iy,tp.I2Iz,tp.I2Ixy,tp.I2Iyz,tp.I2Izx,tp.I2Ex,tp.I2Ey,tp.I2Ez,tp.I2Exy,tp.I2Eyz,tp.I2Ezx,tp.I3Ix,tp.I3Iy,tp.I3Iz,tp.I3Ixy,tp.I3Iyz,tp.I3Izx,tp.I3Ex,tp.I3Ey,tp.I3Ez,tp.I3Exy,tp.I3Eyz,tp.I3Ezx,tp.dtop,tp.d0,tp.d1I,tp.d1E,tp.d2I,tp.d2E,tp.d3I,tp.d3E,tp.d4I,tp.d4E,tp.chainIx,tp.chainEx,tp.n0,tp.n1,tp.n2I,tp.n2E,tp.n3I,tp.n3E,tp.n4I,tp.n4E,tp.n5I,tp.n5E,tp.su,tp.siI,tp.siE,tp.slI,tp.slE,tp.kbuz,tp.kbizI,tp.kbizE,tp.l1,tp.l2I,tp.l2E,tp.l3I,tp.l3E,tp.kw1,tp.kw2I,tp.kw2E,tp.kw3I,tp.kw3E,tp.Y1,tp.Y2I,tp.Y2E,tp.Y3I,tp.Y3E,tp.M11,tp.M21I,tp.M21E,tp.M31I,tp.M31E)

In [46]:
D.shape

(36, 36)

# Modes for the default case 

## What are the default parameters?

In [72]:


pend = tpmodel.pendSS(*[getattr(tp, coeff) for coeff in tpmodel.stdcoeffs]) # generating state space matrices 
modes = tpmodel.eigenmodes(*[getattr(tp, coeff) for coeff in tpmodel.stdcoeffs]) 
print_modes(modes)

4
2
3
None
2
3
0
None
None
4
2
3
0
None
None
0
1
2
0
1
2
0
3
5
None
0
1
None
2
3
0
8
9
4
0
1
None
2
3
None
2
3
None
2
3
4
2
3
0
1
2
4
0
1
None
4
5
2
0
1
9
0
1
4
2
3
None
0
1
None
0
1
None
0
1
4
0
1
None
0
1
4
0
1
9
0
1
9
0
1


,Freq,DOF,Amps,Mode_vis
0,16.469420,"[roll2I, roll2E, roll3I, roll3E, roll1]","[0.53, 0.53, -0.45, -0.45, -0.2]","(0.4444444444444445, 0.4444444444444445)"
1,16.297265,"[roll2I, roll2E, roll3I, roll3E]","[0.53, -0.53, -0.47, 0.47]","(None, None)"
2,14.967516,"[pitch1, z2I, z2E, z3I, z3E]","[1.0, -0.01, 0.01, 0.01, -0.01]","(None, None)"
3,13.539440,"[z2I, z2E, z3I, z3E, z1]","[0.53, 0.53, -0.46, -0.46, -0.14]","(0.30434782608695654, 0.30434782608695654)"
4,12.552206,"[pitch1, z2I, z2E, z3I, z3E]","[-1.0, -0.02, 0.02, 0.02, -0.02]","(None, None)"
5,10.799116,"[roll1, roll3I, roll3E, roll2I, roll2E]","[0.96, -0.21, -0.21, -0.02, -0.02]","(-4.571428571428571, -4.571428571428571)"
6,7.841632,"[z1, z3I, z3E, z2I, z2E]","[-0.94, 0.23, 0.23, 0.07, 0.07]","(-4.08695652173913, -4.08695652173913)"
7,6.066236,"[yaw1, yaw2I, yaw2E, yaw3I, roll3I, yaw3E, rol...","[-0.98, 0.13, 0.13, -0.01, 0.01, -0.01, -0.01]","(98.0, 98.0)"
8,3.703990,"[roll3I, roll3E, roll2I, roll2E, yaw1]","[-0.53, 0.53, -0.47, 0.47, -0.01]","(None, None)"
9,3.064187,"[pitch2I, pitch2E, pitch3I, pitch3E, x1, x2I, ...","[0.61, 0.61, -0.29, -0.29, -0.27, 0.08, 0.08, ...","(None, None)"


# Increase MOI to 2x for the top mass 

In [ ]:
Case1_Tr = pd.DataFrame(data={"Parameters":["I1x","I1y","I1z"], "Values":[0.00235,0.00173,0.00173] })